In [1]:
!pip install openpyxl

In [16]:
import mysql.connector
from mysql.connector import Error
from datetime import date, time
import pandas as pd

# --- 1. Connect to MySQL ---
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""  # <-- fill in your password
)
cursor = conn.cursor()
print("Successfully connected to MySQL.")
print("----------------------------------------")

# --- 2. Create Database ---
cursor.execute("DROP DATABASE IF EXISTS LittleLemonDB")
cursor.execute("CREATE DATABASE LittleLemonDB")
cursor.execute("USE LittleLemonDB")
print("Database 'LittleLemonDB' created and selected successfully.")
print("----------------------------------------")

# --- 3. Drop & Create All Tables ---
cursor.execute("DROP TABLE IF EXISTS Orders")
cursor.execute("DROP TABLE IF EXISTS Bookings")
cursor.execute("DROP TABLE IF EXISTS Customers")
cursor.execute("DROP TABLE IF EXISTS Employees")

cursor.execute("""
CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY AUTO_INCREMENT,
    FirstName VARCHAR(100),
    LastName VARCHAR(100),
    Role VARCHAR(100),
    AnnualSalary INT
)
""")
cursor.execute("""
CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY AUTO_INCREMENT,
    FirstName VARCHAR(100),
    LastName VARCHAR(100),
    Email VARCHAR(255),
    Phone VARCHAR(50)
)
""")
cursor.execute("""
CREATE TABLE Bookings (
    BookingID INT PRIMARY KEY AUTO_INCREMENT,
    TableNo INT,
    BookingDate DATE,
    BookingSlot TIME,
    CustomerID INT,
    EmployeeID INT,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID)
)
""")
cursor.execute("""
CREATE TABLE Orders (
    OrderID INT PRIMARY KEY AUTO_INCREMENT,
    BookingID INT,
    CourseName VARCHAR(255),
    CuisineName VARCHAR(255),
    StarterName VARCHAR(255),
    DessertName VARCHAR(255),
    Drink VARCHAR(255),
    Sides VARCHAR(255),
    Quantity INT,
    BillAmount DECIMAL(10,2),
    FOREIGN KEY (BookingID) REFERENCES Bookings(BookingID)
)
""")
print("All tables created.")
print("----------------------------------------")

# --- 4. Insert Employees ---
employee_rows = [
    ("Mario", "Gollini", "Manager", 70000),
    ("Adrian", "Gollini", "Assistant Manager", 65000),
    ("Giorgos", "Dioudis", "Head Chef", 50000),
    ("Fatma", "Kaya", "Assistant Chef", 45000),
    ("Elena", "Salvai", "Head Waiter", 40000),
    ("John", "Millar", "Receptionist", 35000)
]
cursor.executemany(
    "INSERT INTO Employees (FirstName, LastName, Role, AnnualSalary) VALUES (%s,%s,%s,%s)", 
    employee_rows
)
print("Employees loaded.")

# --- 5. Insert Sample Customers, Bookings, Orders ---
customer_rows = [
    ("John", "Doe", "john@example.com", "555-1010"),
    ("Jane", "Smith", "jane@example.com", "555-2020"),
    ("Alice", "Johnson", "alice@example.com", "555-3030"),
    ("Bob", "Brown", "bob@example.com", "555-4040"),
    ("Priya", "Patel", "priya@example.com", "555-5050"),
    ("Wei", "Chen", "wei@example.com", "555-6060")
]
cursor.executemany(
    "INSERT INTO Customers (FirstName, LastName, Email, Phone) VALUES (%s,%s,%s,%s)",
    customer_rows
)
# Insert Bookings: CustomerID 1..6, assign EmployeeIDs in round-robin, tables 1-6, spread dates/times
booking_rows = [
    (1, date(2025, 8, 30), time(19,0,0), 1, 1),  # TableNo, Date, Slot, CustomerID, EmployeeID
    (2, date(2025, 8, 31), time(12,30,0), 2, 2),
    (3, date(2025, 9, 1), time(18,15,0), 3, 3),
    (4, date(2025, 9, 2), time(20,0,0), 4, 4),
    (5, date(2025, 9, 2), time(9,45,0), 5, 5), # ===== FIX: Changed 09 to 9 =====
    (6, date(2025, 9, 3), time(14,30,0), 6, 6),
]
cursor.executemany(
    "INSERT INTO Bookings (TableNo, BookingDate, BookingSlot, CustomerID, EmployeeID) VALUES (%s,%s,%s,%s,%s)",
    booking_rows
)

# Insert Orders for Bookings 1-6 (simple sample menu)
order_rows = [
    (1, "Dinner", "Italian", "Bruschetta", "Tiramisu", "Red Wine", "Garlic Bread", 2, 45.50),
    (2, "Lunch", "Mexican", "Nachos", "Churros", "Margarita", "Fries", 1, 25.00),
    (3, "Breakfast", "American", "Pancakes", "Muffin", "Coffee", "Bacon", 3, 30.75),
    (4, "Dinner", "Indian", "Samosa", "Gulab Jamun", "Lassi", "Rice", 2, 40.00),
    (5, "Lunch", "Chinese", "Spring Roll", "Fortune Cookie", "Green Tea", "Noodles", 4, 55.20),
    (6, "Dinner", "Greek", "Dolma", "Baklava", "Ouzo", "Pita Bread", 2, 48.90)
]
cursor.executemany("""
INSERT INTO Orders (BookingID, CourseName, CuisineName, StarterName, DessertName, Drink, Sides, Quantity, BillAmount)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
""", order_rows)
conn.commit()
print("Sample data loaded.")
print("----------------------------------------")

# --- 6. Import Orders from Excel into schema, with new customer + booking for each row ---
file_path = "/Users/geoffhawksworth/Desktop/Coursera/Meta/LittleLemon_data.xlsx"
try:
    df = pd.read_excel(file_path)
    df.columns = df.columns.str.strip()
    df.rename(columns={
        "Order ID": "OrderID",
        "Customer Name": "CustomerName",
        "Course Name": "CourseName",
        "Cuisine Name": "CuisineName",
        "Starter Name": "StarterName",
        "Desert Name": "DessertName",
        "Drink": "Drink",
        "Sides": "Sides",
        "Quantity": "Quantity",
        "Sales": "BillAmount"
    }, inplace=True)
    db_columns = ['OrderID', 'CustomerName', 'CourseName', 'CuisineName',
                  'StarterName', 'DessertName', 'Drink', 'Sides', 'Quantity', 'BillAmount']
    df_filtered = df[db_columns]
    for _, row in df_filtered.iterrows():
        names = row['CustomerName'].split(" ", 1)
        first_name = names[0] # ===== FIX: Correctly get the first name =====
        last_name = names[1] if len(names) > 1 else ""
        cursor.execute("""
            INSERT INTO Customers (FirstName, LastName) VALUES (%s,%s)
        """, (first_name, last_name))
        customer_id = cursor.lastrowid
        # Assign a table and employee in round robin, date/times are made up
        table_no = (customer_id % 10) + 1
        emp_id = ((customer_id-1)%6)+1
        bkdate = date(2025,9,5)
        slots = [time(12,0,0), time(13,30,0), time(18,45,0), time(20,0,0), time(10,15,0)]
        slot = slots[customer_id % len(slots)]
        cursor.execute("""
            INSERT INTO Bookings (TableNo, BookingDate, BookingSlot, CustomerID, EmployeeID)
            VALUES (%s, %s, %s, %s, %s)
            """, (table_no, bkdate, slot, customer_id, emp_id))
        booking_id = cursor.lastrowid
        cursor.execute("""
            INSERT INTO Orders (BookingID, CourseName, CuisineName, StarterName,
                                DessertName, Drink, Sides, Quantity, BillAmount)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """, (
            booking_id,
            row['CourseName'],
            row['CuisineName'],
            row['StarterName'],
            row['DessertName'],
            row['Drink'],
            row['Sides'],
            int(row['Quantity']),
            float(row['BillAmount'])
        ))
    conn.commit()
    print("Excel orders inserted (normalized).")
except FileNotFoundError:
    print(f"Excel file not found: {file_path}")
except Exception as e:
    print("Error importing Excel:", e)
print("----------------------------------------")

# --- 7. Stored Procedures (Create/Replace) ---
cursor.execute("DROP PROCEDURE IF EXISTS GetMaxQuantity")
cursor.execute("""
CREATE PROCEDURE GetMaxQuantity()
BEGIN
    SELECT MAX(Quantity) AS MaxOrderQuantity FROM Orders;
END
""")

cursor.execute("DROP PROCEDURE IF EXISTS ManageBooking")
cursor.execute("""
CREATE PROCEDURE ManageBooking(IN booking_date DATE, IN table_no INT)
BEGIN
    DECLARE booking_count INT;
    SELECT COUNT(*) INTO booking_count
    FROM Bookings
    WHERE BookingDate = booking_date AND TableNo = table_no;
    IF booking_count > 0 THEN
        SELECT CONCAT('Table ', table_no, ' is already booked on ', booking_date) AS 'Booking Status',
               CONCAT(c.FirstName, ' ', c.LastName) AS CustomerName,
               CONCAT(e.FirstName, ' ', e.LastName) AS EmployeeName
        FROM Bookings b
        JOIN Customers c ON b.CustomerID = c.CustomerID
        JOIN Employees e ON b.EmployeeID = e.EmployeeID
        WHERE b.BookingDate = booking_date AND b.TableNo = table_no;
    ELSE
        SELECT CONCAT('Table ', table_no, ' is available on this date.') AS 'Booking Status';
    END IF;
END
""")

cursor.execute("DROP PROCEDURE IF EXISTS AddBooking")
cursor.execute("""
CREATE PROCEDURE AddBooking(
    IN table_no INT,
    IN booking_date DATE,
    IN booking_slot TIME,
    IN customer_firstname VARCHAR(100),
    IN customer_lastname VARCHAR(100),
    IN emp_id INT
)
BEGIN
    DECLARE new_customer_id INT;
    INSERT INTO Customers (FirstName, LastName) VALUES (customer_firstname, customer_lastname);
    SET new_customer_id = LAST_INSERT_ID();
    INSERT INTO Bookings (TableNo, BookingDate, BookingSlot, CustomerID, EmployeeID)
    VALUES (table_no, booking_date, booking_slot, new_customer_id, emp_id);
END
""")

cursor.execute("DROP PROCEDURE IF EXISTS UpdateBooking")
cursor.execute("""
CREATE PROCEDURE UpdateBooking(IN booking_id INT, IN new_table_no INT)
BEGIN
    UPDATE Bookings SET TableNo = new_table_no WHERE BookingID = booking_id;
END
""")

cursor.execute("DROP PROCEDURE IF EXISTS CancelBooking")
cursor.execute("""
CREATE PROCEDURE CancelBooking(IN booking_id INT)
BEGIN
    DELETE FROM Orders WHERE BookingID = booking_id;
    DELETE FROM Bookings WHERE BookingID = booking_id;
END
""")
print("Stored procedures created.")
print("----------------------------------------")

# --- 8. Close Connection ---
if conn.is_connected():
    cursor.close()
    conn.close()
    print("Database setup completed successfully and connection is closed.")



Successfully connected to MySQL.
----------------------------------------
Database 'LittleLemonDB' created and selected successfully.
----------------------------------------
All tables created.
----------------------------------------
Employees loaded.
Sample data loaded.
----------------------------------------
Excel orders inserted (normalized).
----------------------------------------
Stored procedures created.
----------------------------------------
Database setup completed successfully and connection is closed.
